In [1]:
import pandas as pd
# Population data
pop_data = pd.read_excel('../population_data_UN.xlsx', index_col=0, header = 1)
pop_data = pop_data[["ISO3 Alpha-code",'Total Population, as of 1 January (thousands)']]
pop_data = pop_data.set_index("ISO3 Alpha-code")
country_codes = pd.read_csv('../countries_codes_and_coordinates.csv' , sep = ',', index_col = 0)

In [2]:
data = pd.read_csv('./results/optimization_results_world_grid.csv')
data = data.set_index('Country')
data = data.drop(['Antarctica'], axis =0)
for col in data.columns[1:]: 
    data[f'{col}_normalized']=data[col]/data['mean_load']
    
for c in data.index.to_list(): 
    results = pd.read_pickle(f'results/{c}/optimization_results.pickle')
    iso_code = country_codes.loc[c,'Alpha-3 code' ].split(' ')[1]
    pop = pop_data.loc[iso_code]
    pop=pop.iloc[0]
    data.loc[c, 'capacity_factor_wind'] = results['E_wind']/data.loc[c, 'P_wind']/8760
    data.loc[c, 'capacity_factor_pv'] = results['E_pv']/data.loc[c, 'P_pv']/8760
    data.loc[c, 'capacity_factor_dispatch'] = results['E_dispatch']/data.loc[c, 'P_dispatch']/8760
    data.loc[c, 'share_dispatchable'] = results['E_dispatch']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data.loc[c, 'share_wind']=results['E_wind']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data.loc[c, 'share_pv']=results['E_pv']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data.loc[c, 'carbon_impact']=results['E_wind']*11+results['E_pv']*45/pop/1000


In [19]:
for c in data.index.to_list(): 
    results = pd.read_pickle(f'results/{c}/optimization_results.pickle')
    iso_code = country_codes.loc[c,'Alpha-3 code' ].split(' ')[1]
    pop = pop_data.loc[iso_code]
    pop=pop.iloc[0]
    data.loc[c, 'E_wind'] = results['E_wind']
    data.loc[c, 'E_pv'] = results['E_pv']
    data.loc[c, 'E_dispatch'] = results['E_dispatch']
    data.loc[c, 'carbon_impact']=results['E_wind']*11+results['E_pv']*45+results['E_dispatch']*440

In [20]:
data['carbon_impact'].sum()/10**12 #Gt

1.9327125138644856

In [23]:
data['E_dispatch'].sum()*500/10**12

1.4349581385059869